In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
  classification_report, 
  confusion_matrix, 
  roc_auc_score, 
  accuracy_score
)

import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)

### Aprendizaje Semi-Supervisado con Datos Parcialmente Etiquetados

**Breast Cancer Wisconsin Dataset**: 

In [ ]:
data = load_breast_cancer()
df = pd.DataFrame(data.data, columns=data.feature_names)
df['target'] = data.target  # 0: maligno, 1: benigno

print(f"""Información del Dataset:
- Dimensión: {df.shape}
- Cantidad de Características: {df.shape[1]-1}""")

In [ ]:
# Distribución de etiquetas y muestra de los datos
display(df['target'].value_counts())
display(df.head())

In [ ]:
# Información del dataset
display(df.info())

### Simulación de Datos Semi-Etiquetados
Para simular un escenario semi-supervisado, se va a marcar algunas muestras como no etiquetadas ($-1$)

In [ ]:
np.random.seed(42)  # para reproducibilidad

# Crear una copia de la variable objetivo original
df['target_semi'] = df['target'].copy()

# Seleccionar aleatoriamente el 70% de los datos para marcarlos como no etiquetados (-1)
n_samples = len(df) 
n_unlabeled = int(0.7 * n_samples)
unlabeled_idx = np.random.choice(df.index, size=n_unlabeled, replace=False)

# Marcar estos índices como no etiquetados
df.loc[unlabeled_idx, 'target_semi'] = -1

In [ ]:
# Distribución de etiquetas en el escenario semi-supervisado
label_counts = df['target_semi'].value_counts()
display(label_counts)

print(f"""Porcentajes:
- Etiquetados (0/1):   {100 * (1 - 0.7):.1f}%
- No Etiquetados (-1): {100 * 0.7:.1f}%""")

In [ ]:
# Separar datos etiquetados y no etiquetados
labeled_mask = df['target_semi'] != -1
unlabeled_mask = df['target_semi'] == -1

df_labeled = df[labeled_mask].copy()
df_unlabeled = df[unlabeled_mask].copy() 

### Análisis Exploratorio de Datos
- Entender la distribución de los datos
- Comparar distribuciones entre datos etiquetados y no etiquetados
- Analizar correlaciones entre características
- Visualizar distribuciones por clase

#### Distribución de Clases y Análisis de Balance

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 1. Distribución de clases en datos etiquetados
class_counts = df_labeled['target'].value_counts().sort_index()
axes[0].bar(['Maligno (0)', 'Benigno (1)'], class_counts.values, color=['#e74c3c', '#2ecc71'])
axes[0].set_title('Distribución de Clases en Datos Etiquetados', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Cantidad de Muestras', fontsize=12)
axes[0].set_xlabel('Clase', fontsize=12)

# Añadir etiquetas con los valores
for i, v in enumerate(class_counts.values):
  axes[0].text(i, v + 3, str(v), ha='center', fontweight='bold')

# 2. Proporción de datos etiquetados vs no etiquetados
total_samples = len(df)
labeled_pct = len(df_labeled) / total_samples * 100
unlabeled_pct = len(df_unlabeled) / total_samples * 100

categories = ['Etiquetados', 'No Etiquetados']
values = [labeled_pct, unlabeled_pct]
colors = ['#3498db', '#f39c12']

axes[1].bar(categories, values, color=colors)
axes[1].set_title('Proporción de Datos Etiquetados vs No Etiquetados', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Porcentaje (%)', fontsize=12)

# Añadir etiquetas con los valores
for i, v in enumerate(values):
  axes[1].text(i, v + 1, f'{v:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

#### Comparación de Distribuciones de las Características

In [ ]:
# Seleccionar las características más importantes según análisis previo
key_features = [
  'worst radius', 
  'worst texture', 
  'worst perimeter', 
  'worst area', 
  'mean radius', 
  'mean texture', 
  'mean perimeter', 
  'mean area'
]

fig, axes = plt.subplots(2, 4, figsize=(18, 8))
axes = axes.flatten()

for idx, feature in enumerate(key_features):
  if idx < len(axes):
    # Histogramas superpuestos
    axes[idx].hist(df_labeled[feature], alpha=0.5, label='Etiquetados', bins=20, color='blue', density=True)
    axes[idx].hist(df_unlabeled[feature], alpha=0.5, label='No Etiquetados', bins=20, color='orange', density=True)
    axes[idx].set_title(feature.replace('_', ' ').title(), fontsize=10)
    axes[idx].set_xlabel('Valor', fontsize=9)
    axes[idx].set_ylabel('Densidad', fontsize=9)
    
    # Añadir estadísticas
    mean_labeled = df_labeled[feature].mean()
    mean_unlabeled = df_unlabeled[feature].mean()
    axes[idx].axvline(mean_labeled, color='blue', linestyle='--', linewidth=1, alpha=0.7, label=f'Media Et: {mean_labeled:.1f}')
    axes[idx].axvline(mean_unlabeled, color='orange', linestyle='--', linewidth=1, alpha=0.7, label=f'Media NoEt: {mean_unlabeled:.1f}')
    
    if idx == 0:
      axes[idx].legend(fontsize=8, loc='upper right')

# Eliminar ejes vacíos si los hay
for idx in range(len(key_features), len(axes)):
  fig.delaxes(axes[idx])

plt.suptitle('Comparación de Distribuciones: Etiquetados vs No Etiquetados', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

#### Test Estadístico de Similitud de Distribuciones (KS Test)

**Test Estadístico de Kolmogorov-Smirnov** (**KS Test**): Prueba estadística no paramétrica utilizada para comparar distribuciones
- *One-Sample KS Test*: Compara una distribución muestral empírica con una distribución teórica de referencia (normal, exponencial, uniforme, etc.)
- *Two-Sample KS Test*: Compara dos distribuciones empíricas para determinar si provienen de la misma distribución. 

In [ ]:
from scipy import stats

p_value_selected = 0.05
ks_results = []

for feature in key_features:
  stat, p_value = stats.ks_2samp(df_labeled[feature].dropna(), df_unlabeled[feature].dropna())
  ks_results.append({
    'Característica': feature,
    'Estadístico KS': stat,
    'p-value': p_value,
    'Diferencia': 'Significativa' if p_value < p_value_selected else 'No significativa'
  })

ks_df = pd.DataFrame(ks_results)
display(ks_df)
print("Interpretación: p-value < 0.05 indica diferencia significativa entre distribuciones")

#### Matriz de Correlación Completa y Análisis de Multicolinealidad

In [ ]:
# Matriz de correlación para todas las características
plt.figure(figsize=(16, 12))
corr_matrix = df_labeled[data.feature_names].corr()

# Crear máscara para el triángulo superior
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

# Heatmap de correlación
sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": .8},
            annot=False, fmt='.2f')

plt.title('Matriz de Correlación Completa - Datos Etiquetados', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
# Análisis de correlaciones
threshold = 0.9
strong_correlations = []

for i in range(len(corr_matrix.columns)):
  for j in range(i + 1, len(corr_matrix.columns)):
    corr_value = abs(corr_matrix.iloc[i, j])
    if corr_value > threshold:
      strong_correlations.append({
        'Característica 1': corr_matrix.columns[i],
        'Característica 2': corr_matrix.columns[j],
        'Correlación': corr_matrix.iloc[i, j]
      })

if strong_correlations:
  strong_corr_df = pd.DataFrame(strong_correlations)
  print(f"Se encontraron {len(strong_correlations)} pares con correlación fuerte:")
  print(strong_corr_df.sort_values('Correlación', ascending=False).to_string(index=False))

  # Identificar características más correlacionadas
  all_features = list(strong_corr_df['Característica 1']) + list(strong_corr_df['Característica 2'])
  feature_counts = pd.Series(all_features).value_counts()
  print("Características más frecuentemente correlacionadas:")
  for feature, count in feature_counts.head(5).items():
    print(f"- {feature}: {count} correlaciones fuertes")
else:
  print(f"No se encontraron correlaciones con umbral: > {threshold})")

#### Análisis de Outliers y Distribución por Clase

In [ ]:
# Seleccionar características más relevantes para análisis de outliers
outlier_features = ['worst radius', 'worst area', 'worst perimeter', 'mean concavity']

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

for idx, feature in enumerate(outlier_features):
  # Boxplot por clase
  sns.boxplot(x='target', y=feature, data=df_labeled, ax=axes[idx],
              palette={'0': '#e74c3c', '1': '#2ecc71'})
  axes[idx].set_title(f'Distribución de {feature.replace("_", " ").title()} por Clase',
                      fontsize=12, fontweight='bold')
  axes[idx].set_xlabel('Clase (0: Maligno, 1: Benigno)', fontsize=10)
  axes[idx].set_ylabel(feature.replace('_', ' ').title(), fontsize=10)

  # Añadir estadísticas
  stats_by_class = df_labeled.groupby('target')[feature].agg(['mean', 'std', 'min', 'max'])
  axes[idx].text(0.02, 0.98, f'Media 0: {stats_by_class.loc[0, "mean"]:.2f}\n'
                  f'Media 1: {stats_by_class.loc[1, "mean"]:.2f}',
                  transform=axes[idx].transAxes, fontsize=9,
                  verticalalignment='top', bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

plt.suptitle('Análisis de Outliers y Distribución por Clase',
              fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

#### Análisis Cuantitativo de Outliers (Método IQR)

In [ ]:
outlier_summary = []
for feature in outlier_features:
  for target_class in [0, 1]:
    subset = df_labeled[df_labeled['target'] == target_class][feature]

    # Calcular IQR
    Q1 = subset.quantile(0.25)
    Q3 = subset.quantile(0.75)
    IQR = Q3 - Q1

    # Definir límites
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR

    # Contar outliers
    outliers = subset[(subset < lower_bound) | (subset > upper_bound)]
    outlier_pct = len(outliers) / len(subset) * 100

    outlier_summary.append({
      'Característica': feature,
      'Clase': target_class,
      'Muestras': len(subset),
      'Outliers': len(outliers),
      '% Outliers': outlier_pct,
      'Q1': Q1,
      'Q3': Q3,
      'IQR': IQR
    })

outlier_df = pd.DataFrame(outlier_summary)
print("Resumen de outliers por característica y clase:")
display(outlier_df[['Característica', 'Clase', 'Muestras', 'Outliers', '% Outliers']])

# Identificar características con más outliers
outlier_by_feature = outlier_df.groupby('Característica')['% Outliers'].mean().sort_values(ascending=False)
print("Características con mayor porcentaje promedio de outliers:")
for feature, pct in outlier_by_feature.items():
  print(f"- {feature}: {pct:.1f}%")

In [ ]:
df_labeled[data.feature_names].describe()

### Procesamiento de Datos

#### División de Datos en Entrenamiento y Prueba

In [ ]:
X_labeled = df_labeled[data.feature_names]
y_labeled = df_labeled['target']

X_unlabeled = df_unlabeled[data.feature_names]

# Dividir datos etiquetados en Entrenamiento y Prueba 
X_train_labeled, X_test, y_train_labeled, y_test = train_test_split(
  X_labeled, 
  y_labeled, 
  test_size=0.3, 
  random_state=42, 
  stratify=y_labeled
)

print(f"""División de Datos:
- X_train_labeled: {X_train_labeled.shape}
- X_test:          {X_test.shape}
- X_unlabeled:     {X_unlabeled.shape}""")

#### Normalización de Características

In [ ]:
scaler = StandardScaler()
X_train_labeled_scaled = scaler.fit_transform(X_train_labeled)
X_test_scaled = scaler.transform(X_test)
X_unlabeled_scaled = scaler.transform(X_unlabeled)

### Baseline: Modelo Supervisado Tradicional

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# 1. Regresión Logística
lr_baseline = LogisticRegression(random_state=42, max_iter=1000)
lr_baseline.fit(X_train_labeled_scaled, y_train_labeled)

# 2. Random Forest
rf_baseline = RandomForestClassifier(random_state=42, n_estimators=100)
rf_baseline.fit(X_train_labeled_scaled, y_train_labeled)

# Evaluar modelos baseline
def evaluate_model(model, X_test, y_test, model_name):
  y_pred = model.predict(X_test)
  y_proba = model.predict_proba(X_test)[:, 1] if hasattr(model, "predict_proba") else None

  print(f"Evaluación de {model_name}:")
  print("=" * 50)
  print(f"Accuracy: {accuracy_score(y_test, y_pred):.4f}")

  if y_proba is not None:
    print(f"AUC-ROC: {roc_auc_score(y_test, y_proba):.4f}")

  print("Reporte de Clasificación:")
  print(classification_report(y_test, y_pred, target_names=['Maligno', 'Benigno']))

  # Matriz de confusión
  cm = confusion_matrix(y_test, y_pred)
  plt.figure(figsize=(6, 5))
  sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
              xticklabels=['Maligno', 'Benigno'],
              yticklabels=['Maligno', 'Benigno'])
  plt.title(f'Matriz de Confusión - {model_name}')
  plt.ylabel('Verdadero')
  plt.xlabel('Predicho')
  plt.tight_layout()
  plt.show()

  return accuracy_score(y_test, y_pred)

# Evaluar ambos modelos baseline
lr_acc = evaluate_model(lr_baseline, X_test_scaled, y_test, "Logistic Regression Baseline")
rf_acc = evaluate_model(rf_baseline, X_test_scaled, y_test, "Random Forest Baseline")

### Aprendizaje Semi-Supervisado: Self-Training

In [ ]:
from sklearn.semi_supervised import SelfTrainingClassifier

print("\n🎯 MODELO SEMI-SUPERVISADO: Self-Training")
print("=" * 60)

# Usar Logistic Regression como clasificador base para Self-Training
base_classifier = LogisticRegression(random_state=42, max_iter=1000)

# Crear modelo de Self-Training
self_training_model = SelfTrainingClassifier(
  base_classifier,
  # Umbral de confianza para etiquetar datos no etiquetados
  threshold=0.75,
  # Usar umbral de confianza
  criterion='threshold',
  # Seleccionar las 10 mejores muestras en cada iteración
  k_best=10,
  # Máximo de iteraciones
  max_iter=50,
)

# Combinar datos etiquetados y no etiquetados
# Para aplicar Self-Training, es necesario un array con -1 para datos no etiquetados
y_semi = np.concatenate([
  y_train_labeled,                      # Etiquetas conocidas
  np.full(len(X_unlabeled_scaled), -1)  # -1 para no etiquetados
])

X_semi = np.vstack([X_train_labeled_scaled, X_unlabeled_scaled])

print(f"""Datos para Self-Training
- Total de muestras:       {len(X_semi)}
- Muestras etiquetadas:    {len(y_train_labeled)}
- Muestras no etiquetadas: {len(X_unlabeled_scaled)}""")

# Entrenar modelo semi-supervisado
self_training_model.fit(X_semi, y_semi)

# Evaluar el modelo
self_train_acc = evaluate_model(
  self_training_model,
  X_test_scaled,
  y_test,
  "Self-Training Semi-Supervised"
)

print(f"""Métricas del Self-Training:
- Número de Iteraciones Locales:    {self_training_model.n_iter_}
- Rango de Iteraciones por Muestra: [{self_training_model.labeled_iter_.min()}, {self_training_model.labeled_iter_.max()}]

Análisis del Proceso de Pseudo-Etiquetado:
- Muestras Etiquetadas Originalmente:   {(self_training_model.labeled_iter_ == 0).sum()}
- Muestras Etiquetadas Automáticamente: {(self_training_model.labeled_iter_ > 0).sum()}
- Muestras que no fueron Etiquetadas:   {(self_training_model.labeled_iter_ == -1).sum()}
- Condición de Parada:                  {self_training_model.termination_condition_}""")


### Aprendizaje Semi-Supervisado: Label Propagation

In [ ]:
from sklearn.semi_supervised import LabelPropagation

# Crear y entrenar modelo de Label Propagation
label_prop = LabelPropagation(
  kernel='rbf', 
  gamma=0.1, 
  max_iter=1000, 
  n_neighbors=7
)

label_prop.fit(X_semi, y_semi)

# Evaluar el modelo
label_prop_acc = evaluate_model(
  label_prop, 
  X_test_scaled, 
  y_test, 
  "Label Propagation"
)

# Obtener las etiquetas predichas para todos los datos
all_labels = label_prop.transduction_

print(f"Distribución de etiquetas después de Label Propagation:")
unique, counts = np.unique(all_labels, return_counts=True)
for label, count in zip(unique, counts):
  print(f"- Clase {label}: {count} muestras ({100*count/len(all_labels):.1f}%)")


### Comparación con Todos los Modelos

In [ ]:
# Crear tabla comparativa
results = pd.DataFrame({
  'Modelo': [
    'Logistic Regression (Baseline)', 
    'Random Forest (Baseline)',
    'Self-Training (Semi-Supervised)',
    'Label Propagation (Semi-Supervised)'
  ],
  'Accuracy': [lr_acc, rf_acc, self_train_acc, label_prop_acc]
})

results = results.sort_values('Accuracy', ascending=False)
results['Mejora vs Baseline'] = results['Accuracy'] - lr_acc

display(results)

In [ ]:
# Visualización comparativa
fig, ax = plt.subplots(figsize=(10, 6))
bars = ax.barh(results['Modelo'], results['Accuracy'], color=['#2E86AB', '#A23B72', '#F18F01', '#C73E1D'])
ax.set_xlabel('Accuracy')
ax.set_title('Comparación de Modelos: Supervisado vs Semi-Supervisado')
ax.set_xlim([0.8, 1.0])

# Añadir valores en las barras
for bar, acc in zip(bars, results['Accuracy']):
  width = bar.get_width()
  ax.text(width + 0.005, bar.get_y() + bar.get_height()/2, f'{acc:.4f}', ha='left', va='center')

plt.tight_layout()
plt.show()

### Análisis de Predicciones en Datos No-Etiquetados

In [ ]:
# Predecir Etiquetas para datos originalmente No Etiquetados
unlabeled_predictions_st = self_training_model.predict(X_unlabeled_scaled)
unlabeled_predictions_lp = label_prop.predict(X_unlabeled_scaled)

# Distribución de Predicciones
print("Distribución de predicciones para datos originalmente no etiquetados:")
print(f"{'Modelo':<30} {'Clase 0':<10} {'Clase 1':<10}")

for name, preds in [('Self-Training', unlabeled_predictions_st), ('Label Propagation', unlabeled_predictions_lp)]:
  unique, counts = np.unique(preds, return_counts=True)
  counts_dict = dict(zip(unique, counts))
  class0 = counts_dict.get(0, 0)
  class1 = counts_dict.get(1, 0)
  print(f"{name:<30} {class0:<10} {class1:<10}")

# Análisis de Consistencia entre Modelos
print("\nConsistencia entre modelos:")
consistency = np.mean(unlabeled_predictions_st == unlabeled_predictions_lp)
print(f"- Porcentaje de acuerdo entre Self-Training y Label Propagation: {100 * consistency:.2f}%")

# Análisis de Confianza
if hasattr(self_training_model, 'predict_proba'):
  probas_st = self_training_model.predict_proba(X_unlabeled_scaled)
  confidence_st = np.max(probas_st, axis=1)

  print(f"\nAnálisis de confianza (Self-Training):")
  print(f"- Confianza promedio: {confidence_st.mean():.4f}")
  print(f"- Confianza mínima:   {confidence_st.min():.4f}")
  print(f"- Confianza máxima:   {confidence_st.max():.4f}")

  # Visualización de distribución de confianza
  plt.figure(figsize=(10, 6))
  plt.hist(confidence_st, bins=20, alpha=0.7, color='#2E86AB', edgecolor='black')
  plt.axvline(x=0.75, color='red', linestyle='--', label='Umbral de Self-Training (0.75)')
  plt.xlabel('Confianza de Predicción')
  plt.ylabel('Frecuencia')
  plt.title('Distribución de Confianza en Predicciones (Self-Training)')
  plt.legend()
  plt.grid(True, alpha=0.3)
  plt.tight_layout()
  plt.show()